# **Importing libraries**

In [1]:
import pygame
import numpy as np
import sys
import random
import time
from pygame.locals import *
import os
import math

pygame 2.5.2 (SDL 2.28.3, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Define global constants

In [2]:
WIDTH, HEIGHT = 576, 1024
GRAVITY = 0.25
BIRD_JUMP = 5
PIPE_GAP = 400
PIPE_VELOCITY = 4
BIRD_WIDTH, BIRD_HEIGHT = 64, 48
PIPE_WIDTH, PIPE_HEIGHT = 90, 600
BACKGROUND_COLOR = (0, 102, 204)

## Initialize Pygame and Set Up the Display

In [3]:
# pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Flappy Bird')
clock = pygame.time.Clock()

## Load assets

In [4]:
# Set images for bird and pipe
bird_img = pygame.image.load('bird.png').convert_alpha()
bird_img = pygame.transform.scale(bird_img, (BIRD_WIDTH, BIRD_HEIGHT))
pipe_img = pygame.image.load('pipe.png').convert_alpha()
pipe_img = pygame.transform.scale(pipe_img, (PIPE_WIDTH, PIPE_HEIGHT))

## Making Bird and Pipe Classes

In [5]:
# Bird class
class Bird:
    def __init__(self,screen):
        # Define Bird's variables that you need to initialize
        self.x = 100
        self.screen=screen
        self.y = HEIGHT // 2
        self.velocity = 0
        self.img = bird_img

    def jump(self):
        # Make a function that when called, causes the bird to jump
        self.velocity = -BIRD_JUMP

    def update(self):
        # Make necessary updates in the bird's velocity as well as y coordinate based on gravity (to be updated each frame).
        self.velocity += GRAVITY
        self.y += self.velocity

    def draw(self):
        # Pygame function to draw the bird on screen
        self.screen.blit(self.img, (self.x, self.y))

# Pipe class
class Pipe:
    def __init__(self, x,screen):
        # Define Pipe's variables that are needed to be initialized.
        self.x = x
        self.screen = screen
        self.height = random.randint(200, 600)
        self.top = self.height - PIPE_HEIGHT
        self.bottom = self.height + PIPE_GAP
        self.pipe_top = pygame.transform.flip(pipe_img, False, True)
        self.pipe_bottom = pipe_img
        self.passed = False

    def move(self):
        # Function to update the pipe's coordinates based on PIPE_VELOCITY (to be called each frame, similar to Bird.update).
        self.x -= PIPE_VELOCITY

    def draw(self):
        # Draw both segments of the pipe on screen
        self.screen.blit(self.pipe_top, (self.x, self.top))
        self.screen.blit(self.pipe_bottom, (self.x, self.bottom))


## Collision detection function

In [6]:
def check_collision(bird, pipes):
    # Check if bird hits the top or bottom of the screen
    if bird.y <= 0 or bird.y + BIRD_HEIGHT >= HEIGHT:
        return True

    # Check for collision with pipes
    for pipe in pipes:
        # Check if the bird is within the pipe's x range
        if pipe.x < bird.x + BIRD_WIDTH < pipe.x + PIPE_WIDTH or pipe.x < bird.x < pipe.x + PIPE_WIDTH:
            # Check if the bird hits the top or bottom pipe
            if bird.y < pipe.height or bird.y + BIRD_HEIGHT > pipe.height + PIPE_GAP:
                return True

    return False


## Main Game Loop

In [8]:
def main():
    # Create bird/pipe instances and score counter
    pygame.init()
    score = 0
    game_over = False
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    bird = Bird(screen)
    pipes = [Pipe(WIDTH, screen)]

    # Load font
    font = pygame.font.Font(None, 36)  # None for default font, 36 for font size

    while not game_over:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                game_over = True
                pygame.quit()
                sys.exit()
            # Set an event checker to allow user to jump
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    bird.jump()

        screen.fill(BACKGROUND_COLOR)

        # Update bird
        bird.update()

        # Remove pipes that have moved off the screen
        pipes = [pipe for pipe in pipes if pipe.x + pipe.pipe_top.get_width() > 0]

        # If there are no pipes or the last pipe is far enough, create a new pipe
        if len(pipes) == 0 or pipes[-1].x < WIDTH - PIPE_GAP:
            pipes.append(Pipe(WIDTH, screen))

        # Move and draw pipes
        for pipe in pipes:
            pipe.move()
            pipe.draw()

            # Check if bird passes through pipes
            if pipe.x + pipe.pipe_top.get_width() < bird.x and not pipe.passed:
                pipe.passed = True
                score += 1
                print("Score: ", score)  # Print score or handle it as needed

            # Check for collision
            if check_collision(bird, pipes):
                game_over = True

        # Draw bird
        bird.draw()

        # Render and display score
        score_text = font.render("Score: " + str(score), True, (255, 255, 255))
        screen.blit(score_text, (10, 10))  # Adjust position as needed

        pygame.display.update()
        clock.tick(60)

    pygame.quit()

if __name__ == "__main__":
    main()


Score:  1
Score:  2
Score:  3
Score:  4
Score:  5
Score:  6
Score:  7
Score:  8
